In [275]:
# You need to create a prototype pricing model that can go through
# further validation and testing before being put into production.
# Eventually, this model may be the basis for fully automated quoting to clients,
# but for now, the desk will use it with manual oversight to explore options with the client.

# You should write a function that is able to use the data you created previously to price the contract.
# The client may want to choose multiple dates to inject and withdraw a set amount of gas,
# so your approach should generalize the explanation from before.
# Consider all the cash flows involved in the product.

# The input parameters that should be taken into account for pricing are:

# > Injection dates.
# > Withdrawal dates.
# > The prices at which the commodity can be purchased/sold on those dates.
# > The rate at which the gas can be injected/withdrawn.
# > The maximum volume that can be stored.
# > Storage costs.

# Write a function that takes these inputs and gives back the value of the contract.
# You can assume there is no transport delay and that interest rates are zero.
# Market holidays, weekends, and bank holidays need not be accounted for.
# Test your code by selecting a few sample inputs.

In [276]:
%reset -f
%load_ext autoreload
%autoreload 2
import os, sys
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from price_predict import load_data

ROOT="/content" # ROOT="/content/TaskOne"
data = os.path.join(ROOT, 'data/Nat_Gas.csv')
df = load_data(data)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/content/price_predict.py:19: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  from sklearn.preprocessing import MinMaxScaler, StandardScaler


In [277]:
def calc_storage_cost_and_drain(curr_vol: int, drain_time: int, GAS_RATE_TPUT: int, STORAGE_COST: float) -> tuple:
    """
    as long as tank is not empty, add a day's storage cost, reduce tank volume
    calculate P/L selling the contract outside this function, storage costs inside

    return tuple of (vol_sold, curr_vol, total_cost)
    """
    if curr_vol <= 0 or drain_time <= 0:
        return 0, 0, 0

    initial_vol = curr_vol
    days_used_drain = 0
    # drain the tank
    for i in range(drain_time):
        days_used_drain += 1

        if curr_vol <= GAS_RATE_TPUT:
            curr_vol = 0
            break
        else:
            curr_vol -= GAS_RATE_TPUT

    total_cost = days_used_drain * STORAGE_COST

    vol_sold = initial_vol - curr_vol
    print(f"Took {days_used_drain} days to drain tank by {vol_sold} units costing ${total_cost:.2f} in storage")

    return vol_sold, curr_vol, total_cost

In [278]:
def calc_volume_and_buy(curr_vol: int, fill_time: int, GAS_RATE_TPUT: int, MAX_VOL: int) -> tuple:
    """
    fill the tank as long as it is not full or there are days available

    return tuple of (vol_bought, curr_vol, total_cost)
    """
    if curr_vol >= MAX_VOL or fill_time <= 0:
        return 0, 0, 0

    vol_bought, days_used_fill = 0, 0

    # fill the tank
    for i in range(fill_time):
        days_used_fill += 1

        if curr_vol + GAS_RATE_TPUT < MAX_VOL:
            vol_bought += GAS_RATE_TPUT
            curr_vol += GAS_RATE_TPUT
        else:
            vol_bought += MAX_VOL - curr_vol
            curr_vol = MAX_VOL
            break

    total_cost = days_used_fill * STORAGE_COST
    print(f"Took {days_used_fill} days to fill tank by {vol_bought} units costing ${total_cost:.2f} in storage")

    return vol_bought, curr_vol, total_cost

In [279]:
from price_predict import run_holt_winters

def price_contract(inj_dates: list, wth_dates: list, GAS_RATE_TPUT: int, MAX_VOL: int, STORAGE_COST: float) -> float:
    """
    Inputs:
    inj_dates:     list of injection dates  (sort earliest to latest)
    wth_dates:     list of withdrawal dates (sort earliest to latest)
    GAS_RATE_TPUT: rate at which the gas can be injected/withdrawn
                        defining this as daily rate
    MAX_VOL:       maximum volume that can be stored
    STORAGE_COST:  cost of storage
                        going to define as cost per day to use tank of capacity max_vol

    Calculate:
    # The prices at which the commodity can be purchased/sold on those dates.
    buy_price_i:  price at which the commodity can be purchased on inj_dates[i]
    sell_price_j: price at which the commodity can be sold on      wth_dates[j]
    # calculate via TaskOne : Holt-Winters price prediction model

    Outputs:
    contract_value: value of the contract
    """
    # exit if Injection dates are empty
    if not inj_dates:
        print("No commodity buying occured. Exiting")
        return 0.0

    # convert Injection dates, Withdrawal dates to pd.to_datetime()
    inj_dates = np.array([pd.to_datetime(x, format='%Y-%m-%d') for x in inj_dates])
    wth_dates = np.array([pd.to_datetime(x, format='%Y-%m-%d') for x in wth_dates])

    # sort Injection dates, Withdrawal dates earliest to latest
    inj_dates = np.sort(inj_dates)
    wth_dates = np.sort(wth_dates)

    # validate if dates are in bounds
    if len(inj_dates) != len(wth_dates):
        print('Error: need the same amount of injection and withdrawal dates')
        return 0.0

    # discard any wth_date_j that occurs before first inj_date_i
    first_buy = inj_dates[0]
    trim_idx = np.searchsorted(wth_dates, first_buy)
    wth_dates = wth_dates[trim_idx:]

    # no issue if no selling occurs, will just store a lot and cost interest
    # make sure not to overflow the tank
    contract_value, storage_cost_total = 0.0, 0.0

    # assume gas price locked in on buy day even if it takes longer to fill / withdraw
    # naive way first - (buy, fill, sell, drain), (buy, fill, sell, drain), ...
    txn_count, curr_tank_vol = 0, 0
    prev_sell_date_j, inj_date_i = None, None
    sell_price_j = 0.0

    # if there are the same number of injection dates and withdrawal dates - expected
    for inj_date_i, wth_date_j in zip(inj_dates, wth_dates):
        # calc draining oil from previous sell date
        # we need to see how much we can drain before next purchase occurs
        # FIXME: handle overlap in future

        # calc draining amount and costs with lookahead to next buy date
        if txn_count > 0:

            days_before_next_buy = (inj_date_i - prev_sell_date_j).days
            days_to_empty_tank = int(np.ceil(curr_tank_vol / GAS_RATE_TPUT))

            drain_time = min(days_before_next_buy, days_to_empty_tank)
            # calculate the storage cost of the draining supply
            vol_sold, curr_tank_vol, store_cost_drain = calc_storage_cost_and_drain(
                curr_tank_vol, drain_time, GAS_RATE_TPUT, STORAGE_COST
            )
            storage_cost_total += store_cost_drain
            print(f'storage cost total after sell and drain #{txn_count-1}: ${storage_cost_total:.2f}')

            # P/L from the amount we could sell
            contract_value += sell_price_j * vol_sold
            print(f"The contract is worth ${contract_value:.2f}\n")

        # fill up the tank
        time_duration = (wth_date_j - inj_date_i).days
        print(f'time_duration = {time_duration}')
        print(f'curr_tank_vol before buy #{txn_count}: {curr_tank_vol}')
        # 0 <= {vol_actual, curr_tank_vol} <= max_vol
        vol_bought, curr_tank_vol, buy_cost = calc_volume_and_buy(
            curr_tank_vol, time_duration, GAS_RATE_TPUT, MAX_VOL
        )
        print(f'curr_tank_vol after buy #{txn_count}: {curr_tank_vol}')

        store_cost_fill = time_duration * STORAGE_COST
        storage_cost_total += store_cost_fill
        print(f'storage cost total after buy #{txn_count}: ${storage_cost_total:.2f}')

        # retrieve buy_price_i and sell_price_i from Holt-Winters price pred. model
        buy_price_i =  run_holt_winters(inj_date_i, df)
        sell_price_j = run_holt_winters(wth_date_j, df)
        print(f"{inj_date_i.strftime('%Y-%m-%d')} buy_price  = ${buy_price_i:.2f} from ML prediction")
        print(f"{wth_date_j.strftime('%Y-%m-%d')} sell_price = ${sell_price_j:.2f} from ML prediction")

        contract_value -= buy_price_i * vol_bought

        # for selling, do at beginning of next loop or after looping, to see draining time and costs
        prev_sell_date_j = wth_date_j
        txn_count += 1

    # drain entire tank as no time limit after final sell if # buys == # sells
    if prev_sell_date_j:
        sell_price = run_holt_winters(prev_sell_date_j, df)
        # calc storage cost during final drainage
        drain_time = int(np.ceil(curr_tank_vol / GAS_RATE_TPUT))
        # calculate the storage cost of the draining supply
        vol_sold, curr_tank_vol, store_cost_drain = calc_storage_cost_and_drain(
            curr_tank_vol, drain_time, GAS_RATE_TPUT, STORAGE_COST
        )
        storage_cost_total += store_cost_drain
        print(f'storage cost total after sell and drain #{txn_count-1}: ${storage_cost_total:.2f}')
        contract_value += sell_price * vol_sold

    print(f"\nTotal storage costs: ${storage_cost_total:.2f}")
    contract_value -= storage_cost_total
    print(f"\nThe contract is worth ${contract_value:.2f} with PnL's minus storage costs\n")
    return contract_value


In [280]:
# Sample Inputs 0
# testing all in bounds, buy then sell without overlap
inj_dates = ["2020-10-11", "2021-04-05", "2022-08-09", "2023-10-01", "2024-09-09", ""] # remove "" to test
wth_dates = ["2021-02-02", "2022-06-27", "2023-06-06", "2024-01-24", "2024-10-10"]
GAS_RATE_TPUT = 10000 # per day
MAX_VOL = 100000     # tank total
STORAGE_COST = 100   # per day
contract_value = price_contract(inj_dates, wth_dates, GAS_RATE_TPUT, MAX_VOL, STORAGE_COST)
# print(f"The contract is worth ${contract_value:.2f}\n")

Error: need the same amount of injection and withdrawal dates


In [281]:
# Sample Inputs 1
# testing past dates, in bounds, future dates...buy then sell without overlap
inj_dates = ["2018-10-11", "2020-04-05", "2022-08-09", "2023-10-01", "2024-05-09", ""] # remove "" to test
wth_dates = ["2019-02-02", "2022-06-27", "2023-06-06", "2024-01-24", "2025-03-20"]
GAS_RATE_TPUT = 10000 # per day
MAX_VOL = 100000     # tank total
STORAGE_COST = 100   # per day
contract_value = price_contract(inj_dates, wth_dates, GAS_RATE_TPUT, MAX_VOL, STORAGE_COST)

Error: need the same amount of injection and withdrawal dates


In [282]:
# Sample Inputs 2
# low throughput - won't be able to full fill or empty tank before next buy/sell
# testing past dates, in bounds, future dates...buy then sell without overlap
inj_dates = ["2018-10-11", "2019-04-05", "2022-08-09", "2022-10-01", "2024-05-09"]
wth_dates = ["2019-02-02", "2022-06-27", "2022-09-06", "2023-01-24", "2025-03-20"]
GAS_RATE_TPUT = 1000  # per day
MAX_VOL = 100000     # tank total
STORAGE_COST = 100   # per day
contract_value = price_contract(inj_dates, wth_dates, GAS_RATE_TPUT, MAX_VOL, STORAGE_COST)

time_duration = 114
curr_tank_vol before buy #0: 0
Took 100 days to fill tank by 100000 units costing $10000.00 in storage
curr_tank_vol after buy #0: 100000
storage cost total after buy #0: $11400.00
2018-10-11 buy_price  = $8.64 from ML prediction
2019-02-02 sell_price = $9.88 from ML prediction
Took 62 days to drain tank by 62000 units costing $6200.00 in storage
storage cost total after sell and drain #0: $17600.00
The contract is worth $-251280.55

time_duration = 1179
curr_tank_vol before buy #1: 38000
Took 62 days to fill tank by 62000 units costing $6200.00 in storage
curr_tank_vol after buy #1: 100000
storage cost total after buy #1: $135500.00
2019-04-05 buy_price  = $9.88 from ML prediction
2022-06-27 sell_price = $10.40 from ML prediction
Took 43 days to drain tank by 43000 units costing $4300.00 in storage
storage cost total after sell and drain #1: $139800.00
The contract is worth $-416625.14

time_duration = 28
curr_tank_vol before buy #2: 57000
Took 28 days to fill tank